### ***                 CHỦ ĐỀ:  ỨNG DỤNG MÔ HÌNH HỌC MÁY VÀO DỰ ĐOÁN SỐ ĐIỆN SỬ DỤNG TRONG THÁNG***


# Cách sử dụng trên Google Colab:

---




**Bước 1: Cài đặt kagglehub (chạy cell đầu tiên)**

**Bước 2: Xác thực Kaggle (Ở đây dataset ta chọn electric-power-consumption-data-set là public nên không cần xác thực)**

**Bước 3: Chạy các cell theo thứ tự**


---




In [ ]:
#Cài đặt và import các thư viện cần thiết
!pip install kagglehub[pandas-datasets]
!pip install optuna
!pip install holidays
import optuna
from sklearn.linear_model import Ridge, Lasso, LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import warnings
import holidays
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

# **1. TIỀN XỬ LÝ DỮ LIỆU (PREPROCESSING)**

---


*1.1. Tải và Làm sạch dữ liệu*

---



In [ ]:
# 1.1 Tải dữ liệu
print("\n[1.1] Đang tải dữ liệu từ Kaggle...")
path = kagglehub.dataset_download("uciml/electric-power-consumption-data-set")
df = pd.read_csv(f"{path}/household_power_consumption.txt", sep=';',
                 parse_dates={'datetime': ['Date', 'Time']},
                 infer_datetime_format=True,
                 low_memory=False, na_values=['?'])
df.set_index('datetime', inplace=True)
print(f"✓ Đã tải {len(df)} dòng dữ liệu từ {df.index.min()} đến {df.index.max()}")



---

1.2 Gom nhóm theo ngày

---



In [ ]:
print("\n[1.2] Gom nhóm dữ liệu theo ngày...")
df_daily = df['Global_active_power'].resample('D').sum().to_frame()
print(f"✓ Đã gom thành {len(df_daily)} ngày")



---
1.3 Xử lý dữ liệu thiếu

---




In [ ]:
print("\n[1.3] Xử lý dữ liệu thiếu...")
missing_before = df_daily['Global_active_power'].isna().sum()
df_daily['Global_active_power'] = df_daily['Global_active_power'].interpolate(method='time')
missing_after = df_daily['Global_active_power'].isna().sum()
print(f"✓ Giảm từ {missing_before} → {missing_after} giá trị thiếu")



---
1.4 Xử lý giá trị ngoại lai

---




In [ ]:
print("\n[1.4] Xử lý outliers...")
upper_limit = df_daily['Global_active_power'].quantile(0.99)
outliers_count = (df_daily['Global_active_power'] > upper_limit).sum()
df_daily['Global_active_power'] = df_daily['Global_active_power'].clip(upper=upper_limit)
print(f"✓ Đã xử lý {outliers_count} outliers (> {upper_limit:.2f} kWh)")



---
1.5 Tích hợp ngày lễ

---




In [ ]:
print("\n[1.5] Tích hợp thông tin ngày lễ...")
fr_holidays = holidays.France()
df_daily['is_holiday'] = df_daily.index.map(lambda x: 1 if x in fr_holidays else 0).astype(int)
print(f"✓ Đã đánh dấu {df_daily['is_holiday'].sum()} ngày lễ")



---
1.6 Tích hợp nhiệt độ

---




In [ ]:
print("\n[1.6] Tích hợp nhiệt độ trung bình theo tháng...")
temp_map = {1: 5, 2: 6, 3: 10, 4: 15, 5: 19, 6: 23, 7: 25, 8: 24, 9: 20, 10: 15, 11: 9, 12: 6}
df_daily['temp_mean'] = df_daily.index.month.map(temp_map)
df_daily['temp_deviation'] = df_daily['temp_mean'] - 15
print(f"✓ Phạm vi nhiệt độ: {df_daily['temp_mean'].min()}°C - {df_daily['temp_mean'].max()}°C")

In [ ]:
print(f"\n✓ HOÀN TẤT TIỀN XỬ LÝ")
print(f"Các cột: {df_daily.columns.tolist()}")
print(f"Kích thước: {df_daily.shape}")





---

**2. KHÁM PHÁ DỮ LIỆU TRỰC QUAN (EDA)**
---
2.1. Biểu đồ 1: Phân bổ

---




In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Biểu đồ 1: Phân bổ
sns.histplot(df_daily['Global_active_power'], kde=True, ax=axes[0,0], color='teal')
axes[0,0].set_title('Phân bổ mức tiêu thụ điện hàng ngày')
axes[0,0].set_xlabel('Điện năng (kWh)')

# Biểu đồ 2: Xu hướng
axes[0,1].plot(df_daily['Global_active_power'], alpha=0.3, label='Hàng ngày')
axes[0,1].plot(df_daily['Global_active_power'].rolling(30).mean(),
               color='red', linewidth=2, label='Trung bình 30 ngày')
axes[0,1].set_title('Xu hướng tiêu thụ điện qua thời gian')
axes[0,1].legend()
axes[0,1].set_ylabel('Điện năng (kWh)')

# Biểu đồ 3: Theo tháng
monthly_avg = df_daily.groupby(df_daily.index.month)['Global_active_power'].mean()
axes[1,0].bar(monthly_avg.index, monthly_avg.values, color='steelblue')
axes[1,0].set_title('Tiêu thụ trung bình theo tháng')
axes[1,0].set_xlabel('Tháng')
axes[1,0].set_ylabel('Điện năng (kWh)')
axes[1,0].set_xticks(range(1, 13))

# Biểu đồ 4: Theo ngày trong tuần
weekly_avg = df_daily.groupby(df_daily.index.dayofweek)['Global_active_power'].mean()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1,1].bar(range(7), weekly_avg.values, color='coral')
axes[1,1].set_title('Tiêu thụ trung bình theo ngày trong tuần')
axes[1,1].set_xlabel('Ngày')
axes[1,1].set_ylabel('Điện năng (kWh)')
axes[1,1].set_xticks(range(7))
axes[1,1].set_xticklabels(days)

plt.tight_layout()
plt.show()



---
## **3.FEATURE ENGINEERING (KỸ THUẬT ĐẶC TRƯNG)**

---

3.1 Tạo các đặc trwngg nâng cao

In [ ]:
def create_advance_features(df):

    df = df.copy()

    # Đặc trưng thời gian cơ bản
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)

    # Mã hóa chu kỳ (sin/cos) - FIX: Thêm day_sin/day_cos
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    df['day_sin'] = np.sin(2 * np.pi * df['dayofweek']/7)
    df['day_cos'] = np.cos(2 * np.pi * df['dayofweek']/7)

    # Đặc trưng lịch sử (Lags)
    df['lag_1'] = df['Global_active_power'].shift(1)
    df['lag_7'] = df['Global_active_power'].shift(7)

    # Đặc trưng cửa sổ trượt
    df['rolling_mean_7'] = df['Global_active_power'].rolling(window=7).mean().shift(1)
    df['rolling_std_7'] = df['Global_active_power'].rolling(window=7).std().shift(1)

    # Biến tương tác thông minh
    df['temp_impact'] = df['temp_mean'] * df['lag_1']
    df['energy_diff'] = df['lag_1'] - df['rolling_mean_7']

    return df.dropna()
print("\n[3.1] Tạo các đặc trưng...")
data = create_advance_features(df_daily)
print(f"✓ Đã tạo {len(data.columns)} cột đặc trưng")
print(f"  Kích thước sau khi dropna: {data.shape}")



---
3.2 Ma trận tương quan

---




In [ ]:
print("\n Phân tích tương quan...")
plt.figure(figsize=(14, 10))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1)
plt.title('Ma trận tương quan giữa các đặc trưng', fontsize=14, pad=20)
plt.tight_layout()
plt.show()



---
3.3 Chuẩn hóa lại data dựa trên mức độ quan trọng

---




In [ ]:
print("\n[3.3] Lọc đặc trưng quan trọng...")
target = 'Global_active_power'

# Loại bỏ các biến thừa thãi
features_to_drop = ['temp_deviation', 'month', 'dayofweek']

# Tính tương quan với target
correlations = data.corr()[target].abs().sort_values(ascending=False)
threshold = 0.05
numeric_features = correlations[correlations > threshold].index.tolist()

# Tổng hợp danh sách cuối
selected_features = [f for f in numeric_features
                    if f != target and f not in features_to_drop]

# Đảm bảo giữ các biến chu kỳ
cyclical_features = ['month_sin', 'month_cos', 'day_sin', 'day_cos']
for feat in cyclical_features:
    if feat not in selected_features and feat in data.columns:
        selected_features.append(feat)

print(f"✓ Số đặc trưng được chọn: {len(selected_features)}")
print(f"✓ Danh sách: {selected_features}")

# Chuẩn bị dữ liệu
X = data[selected_features]
y = data[target]

print(f"\n✓ HOÀN TẤT FEATURE ENGINEERING")
print(f"  Shape: X={X.shape}, y={y.shape}")

## **4. CHIA DỮ LIỆU VÀ CHUẨN HÓA**


---




In [ ]:
print("\n" + "="*80)
print("PHẦN 4: CHIA DỮ LIỆU VÀ CHUẨN HÓA")
print("="*80)

n = len(data)
train_end = int(n * 0.7)
val_end = int(n * 0.85)

train_df = data.iloc[:train_end]
val_df = data.iloc[train_end:val_end]
test_df = data.iloc[val_end:]
train_val_df = data.iloc[:val_end]

print(f"\n✓ Chia dữ liệu:")
print(f"  Train:     {len(train_df)} mẫu ({train_df.index.min()} → {train_df.index.max()})")
print(f"  Validation: {len(val_df)} mẫu ({val_df.index.min()} → {val_df.index.max()})")
print(f"  Test:      {len(test_df)} mẫu ({test_df.index.min()} → {test_df.index.max()})")

# Chuẩn hóa
scaler = MinMaxScaler()
X_train = train_df[selected_features]
y_train = train_df[target]
X_train_scaled = scaler.fit_transform(X_train)

X_train_val_scaled = scaler.transform(train_val_df[selected_features])
y_train_val = train_val_df[target]

print(f"\n✓ Đã chuẩn hóa dữ liệu về khoảng [0, 1]")




---
## **5. DỰ BÁO CUỐN CHIẾU (RECURSIVE FORECASTING)**

---



In [ ]:
def recursive_forecast(model, scaler, history_df, selected_features, steps=30):

    current_history = history_df.copy()
    forecasts = []

    fr_holidays = holidays.France()
    temp_map = {1: 5, 2: 6, 3: 10, 4: 15, 5: 19, 6: 23, 7: 25,
                8: 24, 9: 20, 10: 15, 11: 9, 12: 6}

    for step in range(steps):
        # Tạo đặc trưng từ lịch sử
        features = create_advance_features(current_history)
        X_next = features[selected_features].iloc[-1:]

        # Dự báo
        X_next_scaled = scaler.transform(X_next)
        y_pred = max(0, model.predict(X_next_scaled)[0])
        forecasts.append(y_pred)

        # Cập nhật lịch sử cho vòng lặp tiếp theo
        next_date = current_history.index[-1] + pd.Timedelta(days=1)
        is_h = 1 if next_date in fr_holidays else 0
        t_mean = temp_map[next_date.month]
        t_dev = t_mean - 15

        new_row = pd.DataFrame({
            'Global_active_power': [y_pred],
            'is_holiday': [is_h],
            'temp_mean': [t_mean],
            'temp_deviation': [t_dev]
        }, index=[next_date])

        current_history = pd.concat([current_history, new_row])

    return np.array(forecasts)

## **6. THỰC NGHIỆM VÀ TỐI ƯU TỪNG MÔ HÌNH**

---

6.1. Hàm đánh giá mô hình trên Recursive Forecast

---



In [ ]:
def get_recursive_metrics(model, scaler, df_daily, start_idx, steps=30, selected_features=None):

    history_df = df_daily.iloc[start_idx - 60 : start_idx].copy()
    actual_series = df_daily.iloc[start_idx : start_idx + steps]['Global_active_power']
    current_steps = len(actual_series)

    if current_steps == 0:
        return None

    preds = recursive_forecast(model, scaler, history_df, selected_features, steps=current_steps)
    actual = actual_series.values

    # Tính các chỉ số
    rmse = np.sqrt(mean_squared_error(actual, preds))
    mae = mean_absolute_error(actual, preds)
    mape = np.mean(np.abs((actual - preds) / (actual + 1e-10))) * 100
    r2 = r2_score(actual, preds)

    total_actual = actual.sum()
    total_pred = preds.sum()
    total_err_pct = abs(total_actual - total_pred) / (total_actual + 1e-10) * 100

    return {
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'R2': r2,
        'Total_Err_Pct': total_err_pct,
        'Forecasts': preds,
        'Actual': actual_series,
        'Actual_Sum': total_actual,
        'Pred_Sum': total_pred
    }

print("✓ Đã khởi tạo hệ thống dự báo cuốn chiếu")



---



6.2 TỐI ƯU LINEAR REGRESSION

---



In [ ]:
def optimize_linear_regression(df_daily, train_end, val_end, selected_features, n_trials=30):
    def objective(trial):
        reg_type = trial.suggest_categorical('type', ['ridge', 'lasso'])
        alpha = trial.suggest_float('alpha', 0.01, 100.0, log=True)

        if reg_type == 'ridge':
            model = Ridge(alpha=alpha, random_state=42)
        else:
            model = Lasso(alpha=alpha, max_iter=10000, random_state=42)

        model.fit(X_train_scaled, y_train)

        train_res = get_recursive_metrics(model, scaler, df_daily,
                                         start_idx=train_end-30, steps=30,
                                         selected_features=selected_features)
        val_res = get_recursive_metrics(model, scaler, df_daily,
                                       start_idx=train_end,
                                       steps=(val_end - train_end),
                                       selected_features=selected_features)

        if train_res is None or val_res is None:
            return float('inf')

        val_rmse = val_res['RMSE']
        train_rmse = train_res['RMSE']
        gap = abs(val_rmse - train_rmse)

        return val_rmse + 0.3 * gap

    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print(f"✓ Best params: {study.best_params}, Best score: {study.best_value:.4f}")
    return study



---


6.3 TỐI ƯU RANDOM FOREST

---



In [ ]:
def optimize_random_forest(df_daily, train_end, val_end, selected_features, n_trials=30):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'max_depth': trial.suggest_int('max_depth', 5, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'random_state': 42,
            'n_jobs': -1
        }

        model = RandomForestRegressor(**params).fit(X_train_scaled, y_train)

        tr_res = get_recursive_metrics(model, scaler, df_daily,
                                       start_idx=train_end-30, steps=30,
                                       selected_features=selected_features)
        vl_res = get_recursive_metrics(model, scaler, df_daily,
                                       start_idx=train_end,
                                       steps=(val_end - train_end),
                                       selected_features=selected_features)

        if tr_res is None or vl_res is None:
            return 99999

        val_rmse = vl_res['RMSE']
        train_rmse = tr_res['RMSE']
        gap = abs(val_rmse - train_rmse)

        return val_rmse + 0.5 * gap

    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print(f"✓ Best params: {study.best_params}, Best score: {study.best_value:.4f}")
    return study



---


6.4 TỐI ƯU GRADIENT BOOSTING

---



In [ ]:
def optimize_gradient_boosting(df_daily, train_end, val_end, selected_features, n_trials=30):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 800),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 10),
            'random_state': 42
        }

        model = GradientBoostingRegressor(**params).fit(X_train_scaled, y_train)

        tr_res = get_recursive_metrics(model, scaler, df_daily,
                                       start_idx=train_end-30, steps=30,
                                       selected_features=selected_features)
        vl_res = get_recursive_metrics(model, scaler, df_daily,
                                       start_idx=train_end,
                                       steps=(val_end - train_end),
                                       selected_features=selected_features)

        if tr_res is None or vl_res is None:
            return 99999

        val_rmse = vl_res['RMSE']
        train_rmse = tr_res['RMSE']
        gap = abs(val_rmse - train_rmse)

        return val_rmse + 0.4 * gap

    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print(f"✓ Best params: {study.best_params}, Best score: {study.best_value:.4f}")
    return study



---


6.5 TỐI ƯU XGBOOST

---



In [ ]:
def optimize_xgboost(df_daily, train_end, val_end, selected_features, n_trials=30):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10.0, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 5.0, log=True),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
            'random_state': 42,
            'tree_method': 'hist',
            'n_jobs': -1
        }

        model = xgb.XGBRegressor(**params).fit(X_train_scaled, y_train)

        tr_res = get_recursive_metrics(model, scaler, df_daily,
                                       start_idx=train_end-30, steps=30,
                                       selected_features=selected_features)
        vl_res = get_recursive_metrics(model, scaler, df_daily,
                                       start_idx=train_end,
                                       steps=(val_end - train_end),
                                       selected_features=selected_features)

        if tr_res is None or vl_res is None:
            return 99999

        val_rmse = vl_res['RMSE']
        train_rmse = tr_res['RMSE']
        gap = abs(val_rmse - train_rmse)

        return val_rmse + 0.4 * gap

    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print(f"✓ Best params: {study.best_params}, Best score: {study.best_value:.4f}")
    return study

In [ ]:
print("\n⚙️  BẮT ĐẦU QUÁ TRÌNH TỐI ƯU (Vui lòng đợi 5-10 phút)...")
studies = {
    'Linear Regression': optimize_linear_regression(df_daily, train_end, val_end, selected_features, n_trials=30),
    'Random Forest': optimize_random_forest(df_daily, train_end, val_end, selected_features, n_trials=30),
    'Gradient Boosting': optimize_gradient_boosting(df_daily, train_end, val_end, selected_features, n_trials=30),
    'XGBoost': optimize_xgboost(df_daily, train_end, val_end, selected_features, n_trials=30)
}

print("\n✓ HOÀN TẤT TỐI ƯU HÓA TẤT CẢ MÔ HÌNH")

## **7. SO SÁNH VÀ ĐÁNH GIÁ KẾT QUẢ**

---




In [ ]:
def compare_optimized_models(studies_dict, df_daily, train_end, val_end, selected_features):
    """So sánh tất cả các mô hình đã tối ưu"""
    results = {}

    for name, study in studies_dict.items():
        print(f"\n[Đánh giá] {name}...")
        bp = study.best_params

        # Khởi tạo mô hình với tham số tốt nhất
        if name == 'Linear Regression':
            if bp.get('type') == 'ridge':
                model = Ridge(alpha=bp['alpha'], random_state=42)
            else:
                model = Lasso(alpha=bp['alpha'], max_iter=10000, random_state=42)
        elif name == 'Random Forest':
            model = RandomForestRegressor(**bp, random_state=42, n_jobs=-1)
        elif name == 'Gradient Boosting':
            model = GradientBoostingRegressor(**bp, random_state=42)
        else:
            model = xgb.XGBRegressor(**bp, random_state=42, n_jobs=-1)

        # Đánh giá trên Train và Val
        model.fit(X_train_scaled, y_train)
        train_metrics = get_recursive_metrics(model, scaler, df_daily,
                                             start_idx=train_end-30, steps=30,
                                             selected_features=selected_features)
        val_metrics = get_recursive_metrics(model, scaler, df_daily,
                                           start_idx=train_end,
                                           steps=(val_end-train_end),
                                           selected_features=selected_features)

        # Huấn luyện lại với Train+Val để đánh giá Test
        model.fit(X_train_val_scaled, y_train_val)
        test_metrics = get_recursive_metrics(model, scaler, df_daily,
                                            start_idx=val_end, steps=30,
                                            selected_features=selected_features)

        results[name] = {
            'Train': train_metrics,
            'Val': val_metrics,
            'Test': test_metrics,
            'Best_Params': bp,
            'Model': model
        }
        print(f"  ✓ R2_Test={test_metrics['R2']:.4f}, RMSE_Test={test_metrics['RMSE']:.2f}")

    return results

# So sánh các mô hình
final_results = compare_optimized_models(studies, df_daily, train_end, val_end, selected_features)

# Tạo bảng tổng hợp
print("\n" + "="*80)
print("BẢNG TỔNG HỢP KẾT QUẢ")
print("="*80)

summary = []
for name, m in final_results.items():
    summary.append({
        'Model': name,
        'R2_Train': m['Train']['R2'],
        'R2_Val': m['Val']['R2'],
        'R2_Test': m['Test']['R2'],
        'RMSE_Train': m['Train']['RMSE'],
        'RMSE_Val': m['Val']['RMSE'],
        'RMSE_Test': m['Test']['RMSE'],
        'MAE_Test': m['Test']['MAE'],
        'MAPE_Test (%)': m['Test']['MAPE'],
        'Total_Err_Test (%)': m['Test']['Total_Err_Pct']
    })

df_summary = pd.DataFrame(summary).sort_values(by='R2_Test', ascending=False)
display(df_summary)

# Tìm mô hình tốt nhất
best_model_name = df_summary.iloc[0]['Model']
print(f"\n🏆 MÔ HÌNH TỐT NHẤT: {best_model_name}")
print(f"   R² Score: {df_summary.iloc[0]['R2_Test']:.4f}")
print(f"   RMSE: {df_summary.iloc[0]['RMSE_Test']:.2f} kWh")
print(f"   MAE: {df_summary.iloc[0]['MAE_Test']:.2f} kWh")
print(f"   MAPE: {df_summary.iloc[0]['MAPE_Test (%)']:.2f}%")


## **8. TRỰC QUAN HÓA KẾT QUẢ**

---



In [ ]:
best_result = final_results[best_model_name]['Test']
best_model = final_results[best_model_name]['Model']

# Biểu đồ 1: So sánh Dự báo vs Thực tế
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1.1 Đường thời gian
axes[0, 0].plot(best_result['Actual'].index, best_result['Actual'].values,
               label='Thực tế', linewidth=2, marker='o', markersize=4)
axes[0, 0].plot(best_result['Actual'].index, best_result['Forecasts'],
               label=f'Dự báo ({best_model_name})', linewidth=2,
               marker='s', markersize=4, linestyle='--')
axes[0, 0].set_title('So sánh Dự báo vs Thực tế (30 ngày Test)')
axes[0, 0].set_xlabel('Ngày')
axes[0, 0].set_ylabel('Điện năng (kWh)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# 1.2 Scatter plot
axes[0, 1].scatter(best_result['Actual'].values, best_result['Forecasts'], alpha=0.6)
min_val = min(best_result['Actual'].values.min(), best_result['Forecasts'].min())
max_val = max(best_result['Actual'].values.max(), best_result['Forecasts'].max())
axes[0, 1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2)
axes[0, 1].set_title('Scatter Plot: Thực tế vs Dự báo')
axes[0, 1].set_xlabel('Thực tế (kWh)')
axes[0, 1].set_ylabel('Dự báo (kWh)')
axes[0, 1].grid(True, alpha=0.3)

# 1.3 Residuals (Sai số)
residuals = best_result['Actual'].values - best_result['Forecasts']
axes[1, 0].bar(range(len(residuals)), residuals, color=['red' if x < 0 else 'green' for x in residuals])
axes[1, 0].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1, 0].set_title('Residuals (Sai số dự báo)')
axes[1, 0].set_xlabel('Ngày')
axes[1, 0].set_ylabel('Sai số (kWh)')
axes[1, 0].grid(True, alpha=0.3)

# 1.4 Phân bổ residuals
axes[1, 1].hist(residuals, bins=15, edgecolor='black', alpha=0.7)
axes[1, 1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1, 1].set_title('Phân bổ sai số dự báo')
axes[1, 1].set_xlabel('Sai số (kWh)')
axes[1, 1].set_ylabel('Tần suất')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Biểu đồ 2: So sánh hiệu suất các mô hình
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

models = df_summary['Model'].values
r2_scores = df_summary['R2_Test'].values
rmse_scores = df_summary['RMSE_Test'].values
mape_scores = df_summary['MAPE_Test (%)'].values

# 2.1 R² Score
axes[0].barh(models, r2_scores, color='steelblue')
axes[0].set_xlabel('R² Score')
axes[0].set_title('So sánh R² Score')
axes[0].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(r2_scores):
    axes[0].text(v + 0.01, i, f'{v:.4f}', va='center')

# 2.2 RMSE
axes[1].barh(models, rmse_scores, color='coral')
axes[1].set_xlabel('RMSE (kWh)')
axes[1].set_title('So sánh RMSE')
axes[1].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(rmse_scores):
    axes[1].text(v + 0.5, i, f'{v:.2f}', va='center')

# 2.3 MAPE
axes[2].barh(models, mape_scores, color='lightgreen')
axes[2].set_xlabel('MAPE (%)')
axes[2].set_title('So sánh MAPE')
axes[2].grid(True, alpha=0.3, axis='x')
for i, v in enumerate(mape_scores):
    axes[2].text(v + 0.2, i, f'{v:.2f}', va='center')

plt.tight_layout()
plt.show()

# Biểu đồ 3: Feature Importance (nếu là tree-based model)
if best_model_name in ['Random Forest', 'Gradient Boosting', 'XGBoost']:
    print(f"\n[Feature Importance] {best_model_name}")

    if hasattr(best_model, 'feature_importances_'):
        importances = best_model.feature_importances_
        feature_names = selected_features

        # Sắp xếp theo độ quan trọng
        indices = np.argsort(importances)[::-1][:15]  # Top 15

        plt.figure(figsize=(10, 6))
        plt.barh(range(len(indices)), importances[indices], color='teal')
        plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
        plt.xlabel('Độ quan trọng')
        plt.title(f'Top 15 đặc trưng quan trọng nhất ({best_model_name})')
        plt.gca().invert_yaxis()
        plt.grid(True, alpha=0.3, axis='x')
        plt.tight_layout()
        plt.show()


## **9. KẾT LUẬN VÀ KHUYẾN NGHỊ**

---



In [ ]:
print(f"""
 TÓM TẮT KẾT QUẢ:
--------------------------------------------------
Mô hình tốt nhất: {best_model_name}
- R² Score: {df_summary.iloc[0]['R2_Test']:.4f} (Giải thích {df_summary.iloc[0]['R2_Test']*100:.2f}% phương sai)
- RMSE: {df_summary.iloc[0]['RMSE_Test']:.2f} kWh/ngày
- MAE: {df_summary.iloc[0]['MAE_Test']:.2f} kWh/ngày
- MAPE: {df_summary.iloc[0]['MAPE_Test (%)']:.2f}%
- Sai số tổng hóa đơn: {df_summary.iloc[0]['Total_Err_Test (%)']:.2f}%

 PHÂN TÍCH:
--------------------------------------------------
1. Độ chính xác:
   - Sai số trung bình mỗi ngày: ±{df_summary.iloc[0]['MAE_Test']:.2f} kWh
   - Tỷ lệ sai lệch: {df_summary.iloc[0]['MAPE_Test (%)']:.2f}%
   - Với hóa đơn 30 ngày, sai số dự kiến: {df_summary.iloc[0]['Total_Err_Test (%)']:.2f}%

2. Đặc trưng quan trọng:
   - Các lag features (lag_1, lag_7) có ảnh hưởng lớn
   - Đặc trưng rolling window giúp bắt xu hướng
   - Nhiệt độ và ngày lễ có tác động đáng kể

3. So sánh các mô hình:
   - {best_model_name} vượt trội về khả năng dự báo
   - Linear models phù hợp nếu cần tốc độ cao
   - Tree-based models bắt được phi tuyến tính tốt hơn

 HẠN CHẾ:
--------------------------------------------------
1. Dữ liệu nhiệt độ chỉ là proxy (ước lượng theo tháng)
2. Không có thông tin về giá điện theo giờ
3. Recursive forecasting tích lũy sai số theo thời gian
4. Chưa xét các sự kiện đặc biệt (dịch bệnh, khủng hoảng năng lượng)

 HƯỚNG CẢI TIẾN:
--------------------------------------------------
1. Thu thập dữ liệu thời tiết thực tế (API)
2. Thêm dữ liệu giá điện theo khung giờ
3. Sử dụng mô hình LSTM/Prophet cho chuỗi thời gian
4. Kết hợp ensemble của nhiều mô hình
5. Thêm dữ liệu về các sự kiện lớn (COVID-19, lễ hội)
6. Áp dụng cross-validation chuỗi thời gian (Time Series CV)

 ỨNG DỤNG THỰC TẾ:
--------------------------------------------------
- Tối ưu hóa mua điện cho doanh nghiệp
- Dự báo nhu cầu năng lượng cho grid management
- Phát hiện bất thường trong tiêu thụ điện
- Lập kế hoạch bảo trì thiết bị điện
""")

print("\n" + "="*80)
print(" HOÀN TẤT TOÀN BỘ QUY TRÌNH DỰ BÁO")
print("="*80)